In [1]:
# import necessary packages and utils

import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models, transforms, datasets

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

import PIL.Image as Image
from tqdm import tqdm
import pickle

from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as mse
from sklearn import linear_model
from sklearn.model_selection import StratifiedShuffleSplit

from models.vgg16 import *

In [2]:
# config   

data_dir = "../data/petfinder-pawpularity-score/"

num_epochs = 1
num_classes = 2
batch_size = 16
learning_rate = 1e-5

# model_name = vgg16_bn(pretrained=True) # baseline model

In [ ]:
# device

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
display(device)

In [ ]:
# Image Dataset - might need modification later for covariates idk yet

class PetDataset(Dataset):
    def __init__(self, image_filepaths, targets, transform=None):
        self.image_filepaths = image_filepaths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.image_filepaths[idx]
        
        with open(image_filepath, 'rb') as f:
            image = Image.open(f)
            image_rgb = image.convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        image = torch.tensor(image)
        target = torch.tensor(self.targets[idx])
        
        return image, target

In [6]:
# load cvs data into dataframes

def get_dataframe(data_dir, is_train=True):
    
    if is_train:
        image_dir = os.path.join(data_dir, 'train')
        file_path = os.path.join(data_dir, 'train.csv')
    else:
        image_dir = os.path.join(data_dir, 'test')
        file_path = os.path.join(data_dir, 'test.csv')
    
    df = pd.read_csv(file_path)

    # set image filepath
    df['img_file_path'] = df['Id'].apply(lambda x: os.path.join(image_dir, f'{x}.jpg'))
    
    return df

train_df = get_dataframe(data_dir, is_train=True)
test_df = get_dataframe(data_dir, is_train=False)

In [ ]:
# get DataLoader

def load_data(batch_size, is_train=True, use_subset=False):
    """
    return the train dataloader
    """
    
    if is_train:
        df = get_dataframe(data_dir, is_train=True)
        images = np.array(df['file_path'])
        targets = np.array(df['Pawpularity'])
    else:
        df = get_dataframe(data_dir, is_train=False)
        images = np.array(df['file_path'])
        targets = np.zeros_like(images)
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
    ])
    
    dataset = PetDataset(image_filepaths=images, targets=targets, transform=transform)
    
    subse_ind = list(range(500))
    
    data_subset = Subset(dataset, subse_ind)

    # data loader
    data_loader = DataLoader(dataset=data_subset if use_subset else dataset, 
                                batch_size=batch_size,
                                shuffle=True)
    
    return data_loader

train_loader = load_data(batch_size, is_train=True)
test_loader = load_data(batch_size, is_train=False)

In [ ]:
# init model

class ConvNet_v1(nn.Module):
    """
    Simple two-layer CNN with sequential container
    """
    def __init__(self):
        super(ConvNet_v1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(387200, 128),
            nn.ReLU(),
            nn.Linear(128, 1) 
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out).squeeze()
        return out

In [ ]:
# init model

def initialize_model(model, learning_rate, num_classes):
    """
    initialize the model
    define loss function and optimizer and move data to gpu if available
    
    return:
        model, loss function(criterion), optimizer
    """
    
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    return model, criterion, optimizer

model_name = ConvNet_v1()

model, criterion, optimizer = initialize_model(model_name, learning_rate, num_classes)

In [ ]:
# plotting loss curve

def make_plots(step_hist, loss_hist):
    plt.plot(step_hist, loss_hist)
    plt.xlabel('train_iterations')
    plt.ylabel('Loss')
    # plt.title('epoch'+str(epoch+1))
    plt.title(experiment_name)
    plt.savefig(experiment_name)
    plt.clf()

In [ ]:
# train

experiment_name = "simple_run"

def calc_rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).mean())


def train(train_loader, model, criterion, optimizer, num_epochs):
    """
    Move data to GPU memory and train for specified number of epochs
    Also plot the loss function and save it in `Figures/`
    Trained model is saved as `cnn.ckpt`
    """
    # for each training sample
    loss_hist = []
    step_hist = []
    for i, (images, label) in (enumerate(train_loader)):

        model.train()
        train_pred = list()
        train_true = list()

        # move to gpu if available
        images = images.to(device).float()
        label = label.to(device).float()

        # forward pass
        out = model(images)
        loss = torch.sqrt(criterion(out, label))

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i // 5000 == 0:
            train_true += label.cpu().detach().numpy().tolist()
            train_pred += out.cpu().detach().numpy().tolist()

            train_rmse = calc_rmse(np.array(train_pred), np.array(train_true))
            
            step_hist.append(i)
            loss_hist.append(train_rmse)
            print('Train rmse: {}'.format(train_rmse))
            
    
        # plot
        make_plots(step_hist, loss_hist)

torch.save(model.state_dict(), experiment_name+'.ckpt')

In [ ]:
train(train_loader, model, criterion, optimizer, num_epochs)

In [ ]:
# eval

# iterate over test data
print('Making predictions...')
test_pred = []

for test_images, test_labels in test_loader:
    test_images = test_images.to(device).float()
    
    # forward
    out = model(test_images)
    test_pred += out.cpu().detach().numpy().tolist()

# write to file
output = pd.DataFrame({"Id": test_df['Id'], "Pawpularity": test_pred})
output.to_csv('submission.csv', index = False)

In [ ]:
# check output

output_df = pd.read_csv('submission.csv')
output_df